<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [51]:
#import package
!pip install censusgeocode
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data cleaning

In [60]:
data= pd.read_csv("final.csv")
# select the data columns we want 
data = data[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','LivingArea']]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (23,24,25,27,29,30,31,76,92,132,137,139,171,173,197,202,226,227,228,229,253,255,299,300,355,359,360,361,362,379,382,383) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Set Input

In [61]:
# change data type
zip = []
lo = []
la = []
sqt = []

for item in data['PostalCode']:
    item = str(item)
    zip.append(item)
for item in data['Longitude']:
    item = float(item)
    lo.append(item)
for item in data['Latitude']:
    item = float(item)
    la.append(item)
for item in data['LivingArea']:
    item = float(item)
    sqt.append(item)

data['PostalCode'] = zip
data['Longitude'] = lo
data['Latitude'] = la
data['LivingArea'] = sqt

In [62]:
# choose columns status

data = data.loc[data['PropertySubType'] == 'Single Family Residence' ]
data = data.loc[data['PropertyType'] == 'Residential']

# filter the data

In [63]:
# input data address
print('Enter #MLS')
mls = 14730621
print('Enter address')
address = '2836 Meadowbrook Drive'
print('Enter city')
city = 'Dallas'
print('Enter state')
state = 'Texas'
print('Enter zipcode')
zip = '75227'



Enter #MLS
Enter address
Enter city
Enter state
Enter zipcode


# code part

In [64]:
def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, data):
    basic = house_leftSqt
    result = data[(data['Longitude'] >= house_leftLo) & (data['Longitude'] <= house_rightLo) & (data['Latitude'] >= house_leftLa) & (data['Latitude'] <= house_rightLa) & (data['LivingArea'] >= house_leftSqt) & (data['LivingArea'] <= house_rightSqt)]
    if(len(result['Longitude']) >= 6):
        return result
    else:
        house_leftLa -= 0.05
        house_rightLa += 0.05
        house_leftLo -= 0.05
        house_rightLo += 0.05
        if(basic <= 1100):
            house_leftSqt -= 100
            house_rightSqt += 100
        elif(basic <= 2200):
            house_leftSqt -= 200
            house_rightSqt += 200
        else:
            house_leftSqt -= 300
            house_rightSqt += 300
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, data)  
        

#findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, data)

In [65]:
# find value in csv 
res_mls = data.isin([mls]).any().any()
if res_mls :
  print("\nThis value exists in Dataframe")
  # get whole row value
  pulled_value = data.loc[data['ListingId'] == mls]
  #get each value
  dict = pulled_value.to_dict('records')
  #defined number
  house_leftLa = dict[0].get('Latitude')
  house_rightLa = dict[0].get('Latitude') 
  house_leftLo = dict[0].get('Longitude')
  house_rightLo = dict[0].get('Longitude')
  house_leftSqt = dict[0].get('LivingArea')
  house_rightSqt = dict[0].get('LivingArea')

  # get zip
  data = data.loc[data['PostalCode'] == zip ]
  findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, data)


else :
  print("\nThis value does not exists in Dataframe")
  # # limit zipcode range
  
  # if there is no longitude 
  #get Longitude and Latitude

  ##cg = CensusGeocode(benchmark='Public_AR_Current', vintage='Current_Current')
  result = cg.address(address, city = city, state= state,zipcode = zip)
  lo = result[0]['coordinates']['x']
  la = result[0]['coordinates']['y']
  print(result[0]['coordinates'])
  data = data.loc[data['PostalCode'] == zip ]
  new_data = data[(data['Longitude'] > lo - 0.01) & (data['Longitude'] < lo + 0.01)&
               (data['Latitude'] < la + 0.02)& (data['Latitude'] > la - 0.02)]


This value exists in Dataframe


SyntaxError: ignored

In [ ]:
# limit the days
data = data.loc[data['PostalCode'] == zip ]
data = data[data['CumulativeDaysOnMarket'].isin(range(0,90))] 

In [ ]:
# limit square feet

if 'LivingArea' <= 1100:
  data[(data['LivingArea'] > area - 100) & (data['LivingArea'] < area + 100)
elif 'LivingArea' <= 2200:
  data[(data['LivingArea'] > area -200 )& (data['LivingArea'] > area + 200)]
else:
  data[(data['LivingArea'] > area -300 )& (data['LivingArea'] > area + 300)]


In [44]:
#defined number
house_leftLa = dict[0].get('Latitude')
house_rightLa = dict[0].get('Latitude') 
house_leftLo = dict[0].get('Longitude')
house_rightLo = dict[0].get('Longitude')
house_leftSqt = dict[0].get('LivingArea')
house_rightSqt = dict[0].get('LivingArea')